In [1]:
import pandas as pd
import json
import requests
url = 'https://raw.githubusercontent.com/brearenee/NLP-Project/main/dataset/StarTrekDialogue_v2.json'
response = requests.get(url)

##This CodeBlock is thanks to ChatGPT :-) 
if response.status_code == 200:
    json_data = json.loads(response.text)
    lines = []
    characters = []
    episodes = []
  
    # extract the information from the JSON file for the "TNG" series
    for series_name, series_data in json_data.items():
        if series_name == "TNG": 
            for episode_name, episode_data in series_data.items():
                for character_name, character_lines in episode_data.items():
                    for line_text in character_lines:
                        lines.append(line_text)
                        characters.append(character_name)
                        episodes.append(episode_name)
                     
    # Create a DataFrame from the extracted data
    df = pd.DataFrame({
        'Line': lines,
        'Character': characters,
    })

    # Remove duplicate lines, keeping the first occurrence (preserving the original order)
    df = df.drop_duplicates(subset='Line', keep='first')

    # Reset the index of the DataFrame
    df.reset_index(drop=True, inplace=True)

else:
    print(f"Failed to retrieve data. Status code: {response.status_code}")
    
    
##Remove Outliers (Characters with less than 1000 lines)
character_counts = df['Character'].value_counts()
characters_to_remove = character_counts[character_counts < 1000].index
df = df[~df['Character'].isin(characters_to_remove)]

##Print Value Count. 
print(df['Character'].value_counts())


Character
PICARD     10798
RIKER       6454
DATA        5699
LAFORGE     4111
WORF        3185
CRUSHER     2944
TROI        2856
WESLEY      1206
Name: count, dtype: int64


# BERT

In [2]:
#https://www.analyticsvidhya.com/blog/2021/12/multiclass-classification-using-transformers/


#Split the data 
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

#Converting our Character column into Categorical data
encoded_dict = {'PICARD':0,'RIKER':1, 'DATA':2, 'LAFORGE':3, 
                'WORF':4, 'CRUSHER':5, 'TROI':6,'WESLEY':7}
train_df['Character'] = train_df.Character.map(encoded_dict)
val_df['Character'] = val_df.Character.map(encoded_dict)

train_df.head()




,Line,Character
41033,The upper portion of the apparatus seems\r to ...,2
19647,how long before they cross over into Federatio...,1
20640,"You know, this might work. We can't change the...",3
16952,And you conclude because of this that I am imp...,2
11530,I was thinking the same thing about you. In al...,1


In [3]:
val_df.head()

,Line,Character
48560,"If you had to give this feeling a name, what w...",6
47262,I'm sure you all understand that in light of w...,5
7425,"Somehow, and there is limited information on t...",2
29414,But our sensors were malfunctioning. Our probe...,1
33815,The signal ended abruptly at oh four five five...,1


In [4]:
from tensorflow.keras.utils import to_categorical

y_train = to_categorical(train_df.Character)
y_test = to_categorical(val_df.Character)

#We have successfully processed our Sentiment column( target); 
#now, it’s time to process our input text data using a tokenizer.